<a href="https://colab.research.google.com/github/aLehav/MLVideoDescriptionResearch/blob/main/BEATs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

[Link to BEATs Github](https://github.com/microsoft/unilm/tree/master/beats)

In [ ]:
import torch
from Tokenizers import TokenizersConfig, Tokenizers
from BEATs import BEATs, BEATsConfig


# load the pre-trained checkpoints
checkpoint = torch.load('/path/to/tokenizer.pt')

audio_input_16khz = torch.randn(1, 10000)
padding_mask = torch.zeros(1, 10000).bool()

# for a tokenizer
# cfg = TokenizersConfig(checkpoint['cfg'])
# BEATs_tokenizer = Tokenizers(cfg)
# BEATs_tokenizer.load_state_dict(checkpoint['model'])
# BEATs_tokenizer.eval()

# labels = BEATs_tokenizer.extract_labels(audio_input_16khz, padding_mask=padding_mask)


#for pretrained
# cfg = BEATsConfig(checkpoint['cfg'])
# BEATs_model = BEATs(cfg)
# BEATs_model.load_state_dict(checkpoint['model'])
# BEATs_model.eval()

# representation = BEATs_model.extract_features(audio_input_16khz, padding_mask=padding_mask)[0]

